In [1]:
import warnings
warnings.filterwarnings("ignore")
# !pip install ftfy
import ftfy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
from math import exp
from numpy import sign
import sys
print(sys.executable)
from PIL import Image # getting images in notebook
# !pip install gensim
from nltk.stem.snowball import SnowballStemmer
from sklearn.metrics import  classification_report, confusion_matrix, accuracy_score
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from nltk import PorterStemmer
from nltk.tokenize import RegexpTokenizer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# !pip install tensorflow

# !pip install tensorflow_hub

# !pip install bert-for-tf2
# !pip install sentencepiece

C:\Users\SHWETANK VERMA\Anaconda3\latest\python.exe


[nltk_data] Downloading package stopwords to C:\Users\SHWETANK
[nltk_data]     VERMA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\SHWETANK
[nltk_data]     VERMA\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# from tensorflow import keras 
import tensorflow.keras as keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, MaxPooling1D
from tensorflow.keras.preprocessing.text import Tokenizer
#from keras_preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import plot_model
import re
from tqdm import tqdm
import tensorflow as tf
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow_hub as hub

In [3]:
train=pd.read_csv(r'C:\Users\SHWETANK VERMA\Documents\Mlstuff\Major-1\Datasets\train.csv')

In [4]:
train.sample(3)

,PID,Text_data,Label
8302,train_pid_8303,"Parents, Indian parents... Help [20M] : So, my...",severe
8447,train_pid_8448,What can you do? : A successful person cannot ...,severe
7347,train_pid_7348,My friend told me to kms : [removed],not depression


In [5]:
test=pd.read_csv(r'C:\Users\SHWETANK VERMA\Documents\Mlstuff\Major-1\Datasets\dev.csv')

In [6]:
print("Training set has {} rows and {} columns.".format(train.shape[0], train.shape[1]))
print("Test set has {} rows and {} columns.".format(test.shape[0], test.shape[1]))

print()
print(train.columns)
print(test.columns)

Training set has 8891 rows and 3 columns.
Test set has 4496 rows and 3 columns.

Index(['PID', 'Text_data', 'Label'], dtype='object')
Index(['PID', 'Text data', 'Label'], dtype='object')


In [7]:
Sentiment = {'moderate': 1,'not depression': 0,'severe':2}
train.Label = [Sentiment[item] for item in train.Label]
test.Label= [Sentiment[item] for item in test.Label]

In [8]:
train.sample(3)

,PID,Text_data,Label
1039,train_pid_1040,Worst day of the year for me : Every year I’m ...,1
3466,train_pid_3467,How can I open up to my parents : Hi I want to...,1
452,train_pid_453,Why the fuck am i said : Why the fuck am i sad...,1


In [9]:
test.sample(3)


,PID,Text data,Label
1115,dev_pid_1116,Anhedonhia is destroying me : Every fucking da...,1
2024,dev_pid_2025,I felt happy for a day : I felt like no one ha...,1
3656,dev_pid_3657,Anyone Else Wish for A Fatal Disease?? : [remo...,0


In [10]:
import string
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

# NLTK Tweet Tokenizer for now
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer(strip_handles=True)

corpus = []

# clean up text
def clean_text(text):
    """
    Copied from other notebooks
    """
    # expand acronyms
    
    # special characters
    text = re.sub(r"\x89Û_", "", text)
    text = re.sub(r"\x89ÛÒ", "", text)
    text = re.sub(r"\x89ÛÓ", "", text)
    text = re.sub(r"\x89ÛÏWhen", "When", text)
    text = re.sub(r"\x89ÛÏ", "", text)
    text = re.sub(r"China\x89Ûªs", "China's", text)
    text = re.sub(r"let\x89Ûªs", "let's", text)
    text = re.sub(r"\x89Û÷", "", text)
    text = re.sub(r"\x89Ûª", "", text)
    text = re.sub(r"\x89Û\x9d", "", text)
    text = re.sub(r"å_", "", text)
    text = re.sub(r"\x89Û¢", "", text)
    text = re.sub(r"\x89Û¢åÊ", "", text)
    text = re.sub(r"fromåÊwounds", "from wounds", text)
    text = re.sub(r"åÊ", "", text)
    text = re.sub(r"åÈ", "", text)
    text = re.sub(r"JapÌ_n", "Japan", text)    
    text = re.sub(r"Ì©", "e", text)
    text = re.sub(r"å¨", "", text)
    text = re.sub(r"SuruÌ¤", "Suruc", text)
    text = re.sub(r"åÇ", "", text)
    text = re.sub(r"å£3million", "3 million", text)
    text = re.sub(r"åÀ", "", text)
    
    # emojis
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    
    """
    Our Stuff
    """
    # remove numbers
    text = re.sub(r'[0-9]', '', text)
    
    # remove punctuation and special chars (keep '!')
    for p in string.punctuation.replace('!', ''):
        text = text.replace(p, '')
        
    # remove urls
    text = re.sub(r'http\S+', '', text)
    
    # tokenize
    text = tknzr.tokenize(text)
    
    # remove stopwords
    text = [w.lower() for w in text if not w in stop_words]
    corpus.append(text)
    
    # join back
    text = ' '.join(text)
    
    return text

In [11]:
%%time
train['Text_data'] = train['Text_data'].apply(lambda s: clean_text(s))
test['Text data'] = test['Text data'].apply(lambda s: clean_text(s))

# see some cleaned data
train.sample(10)

Wall time: 30.1 s


,PID,Text_data,Label
2308,train_pid_2309,worst year depression crisis im happy ended af...,1
3286,train_pid_3287,being okay temprorary happiness ive learnt oka...,1
8195,train_pid_8196,my problem i know sound fucking egotistic text...,2
5826,train_pid_5827,a letter daughter depression dear daughter i h...,1
2342,train_pid_2343,less hours i dont feel ready everything especi...,1
2811,train_pid_2812,fuck titles i ’ depressed i feel fucking worth...,1
3082,train_pid_3083,i never sad new years yesterday i literally br...,1
894,train_pid_895,just exhausted no matter much right amount sle...,1
263,train_pid_264,i hate life its everyone fun im sitting alone ...,1
4997,train_pid_4998,why i like there deep feeling sadness seems po...,1


In [12]:
texts = train['Text_data'].to_numpy()
word_freq = {}

for text in texts:
    for word in text.split():
        word_freq[word] = word_freq.get(word, 0) + 1

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_SEQUENCE_LENGTH = 40

tokenizer = Tokenizer()

tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
num_words = len(word_index) + 1
print('Found %s unique tokens.' % (num_words - 1))

# pad 
data = pad_sequences(
    sequences, 
    maxlen=MAX_SEQUENCE_LENGTH,
    padding='post', 
    truncating='post'
)

labels = train['Label'].to_numpy()
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Found 13266 unique tokens.
Shape of data tensor: (8891, 40)
Shape of label tensor: (8891,)


In [14]:
x_train = data
y_train = labels

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.metrics import roc_auc_score

vectorizer = CountVectorizer()
x_train_vectorized = vectorizer.fit_transform(train['Text_data'])

# print vocabulary
print(vectorizer.get_feature_names()[2500:2600])

['courageous', 'course', 'courses', 'coursework', 'court', 'courtesy', 'cousin', 'cousindads', 'cousins', 'cousy', 'cover', 'coverage', 'covered', 'covering', 'covers', 'covid', 'cow', 'coward', 'cowardice', 'cowards', 'cower', 'cowered', 'coworker', 'coworkers', 'coz', 'cozy', 'cps', 'cptsd', 'crab', 'crack', 'cracked', 'cracking', 'cracks', 'cradle', 'cradled', 'craft', 'crafts', 'craig', 'craigslist', 'cramps', 'crap', 'crappier', 'crappy', 'crash', 'crashes', 'crashing', 'crave', 'craved', 'craving', 'cravings', 'crawl', 'crawled', 'crawling', 'crazed', 'craziest', 'crazy', 'crazyo', 'cream', 'create', 'created', 'creates', 'creating', 'creation', 'creations', 'creative', 'creativity', 'creator', 'creators', 'creature', 'creatures', 'credit', 'creep', 'creeped', 'creeping', 'creeps', 'creepy', 'crept', 'crevice', 'crib', 'crickets', 'cried', 'cries', 'crieswhy', 'crime', 'crimes', 'criminal', 'cringe', 'cringey', 'cringing', 'cringy', 'cripple', 'crippled', 'cripples', 'crippling',

In [16]:
x = train["Text_data"]
y = train["Label"]

In [17]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split

from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [18]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

weather_encoded = le.fit_transform(x)


label=le.fit_transform(y)

In [19]:
features = list(weather_encoded)

In [25]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)

In [28]:
import numpy as np
from keras.datasets import imdb
import tensorflow 
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [30]:
from keras.layers import *
from keras.models import *
import keras.backend as K #for some advanced functions 

In [32]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(train['Text_data'].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 13266 unique tokens.


In [74]:
X = tokenizer.texts_to_sequences(train['Text_data'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (8891, 250)


In [75]:
Y = pd.get_dummies(train['Label']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (8891, 3)


In [35]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.10, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(8001, 250) (8001, 3)
(890, 250) (890, 3)


In [37]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 5
batch_size = 64

history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/5
113/113 [==============================] - 135s 1s/step - loss: 0.7302 - accuracy: 0.7163 - val_loss: 0.4715 - val_accuracy: 0.8427
Epoch 2/5
113/113 [==============================] - 129s 1s/step - loss: 0.3360 - accuracy: 0.8790 - val_loss: 0.3759 - val_accuracy: 0.8777
Epoch 3/5
113/113 [==============================] - 135s 1s/step - loss: 0.1899 - accuracy: 0.9372 - val_loss: 0.3599 - val_accuracy: 0.8864
Epoch 4/5
113/113 [==============================] - 133s 1s/step - loss: 0.1527 - accuracy: 0.9517 - val_loss: 0.3805 - val_accuracy: 0.8851
Epoch 5/5
113/113 [==============================] - 135s 1s/step - loss: 0.0932 - accuracy: 0.9700 - val_loss: 0.4346 - val_accuracy: 0.8889


In [38]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

28/28 [==============================] - 2s 70ms/step - loss: 0.5352 - accuracy: 0.8663
Test set
  Loss: 0.535
  Accuracy: 0.866


In [ ]:
#using glove 

In [76]:
X_train, X_test,Y_train, Y_test = train_test_split(X,Y, test_size=0.1, random_state = 42)

In [77]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [42]:
def read_glove_vector(glove_vec):
  with open(glove_vec, 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
      w_line = line.split()
      curr_word = w_line[0]
      word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)



  return word_to_vec_map

In [46]:
word_to_vec_map = read_glove_vector('glove.twitter.27B.100d.txt')

maxLen = 150

In [64]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['moon'].shape[0]
print(embed_vector_len)
emb_matrix = np.array([], dtype=np.int64)  
emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
  #print(word,index)
  embedding_vector = word_to_vec_map.get(word)
  #print(embedding_vector)
  if embedding_vector is not None and embedding_vector.size != 0:
    emb_matrix[index,:] = embedding_vector
  if(index==12430):
    break
embedding_layer = Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False)

100


In [65]:
def imdb_rating(input_shape):

  X_indices = Input(input_shape)

  embeddings = embedding_layer(X_indices)

  X = LSTM(128, return_sequences=True)(embeddings)

  X = Dropout(0.6)(X)

  X = LSTM(128, return_sequences=True)(X)

  X = Dropout(0.6)(X)

  X = LSTM(128)(X)

  X = Dense(1, activation='sigmoid')(X)

  model = Model(inputs=X_indices, outputs=X)

  return model

In [78]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=250, padding='post')

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [79]:
Y_train.shape

(7112, 3)

In [80]:
adam = keras.optimizers.Adam(learning_rate = 0.0001)
model.compile(optimizer=adam, loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train_indices, Y_train, batch_size=64, epochs=15)

Epoch 1/15
112/112 [==============================] - 139s 1s/step - loss: 0.5417 - accuracy: 0.6457
Epoch 2/15
112/112 [==============================] - 142s 1s/step - loss: 0.5024 - accuracy: 0.6727
Epoch 3/15
112/112 [==============================] - 142s 1s/step - loss: 0.4991 - accuracy: 0.6744
Epoch 4/15
112/112 [==============================] - 146s 1s/step - loss: 0.4948 - accuracy: 0.6758
Epoch 5/15
112/112 [==============================] - 145s 1s/step - loss: 0.4932 - accuracy: 0.6774
Epoch 6/15
112/112 [==============================] - 141s 1s/step - loss: 0.4916 - accuracy: 0.6781
Epoch 7/15
112/112 [==============================] - 145s 1s/step - loss: 0.4901 - accuracy: 0.6796
Epoch 8/15
112/112 [==============================] - 127s 1s/step - loss: 0.4877 - accuracy: 0.6807
Epoch 9/15
112/112 [==============================] - 138s 1s/step - loss: 0.4864 - accuracy: 0.6808
Epoch 10/15
112/112 [==============================] - 143s 1s/step - loss: 0.4831 - accura

In [81]:
accr = model.evaluate(X_test,Y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

56/56 [==============================] - 6s 91ms/step - loss: 0.6125 - accuracy: 0.6301
Test set
  Loss: 0.612
  Accuracy: 0.630


In [84]:
import fasttext
import fasttext.util

In [85]:
ft = fasttext.load_model('cc.en.300.bin')

In [87]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
stemmer = PorterStemmer()
lemmatizer= WordNetLemmatizer()

In [130]:
max_features = 5000
maxlen = 1400
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import BatchNormalization
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
embedding_vector_length = 50
embedding_matrix_fasttext = np.zeros((max_features + 1, embedding_vector_length))
print(embedding_matrix_fasttext.shape)

(5001, 50)


In [131]:
token = tf.keras.preprocessing.text.Tokenizer(num_words=max_features)
word_index = token.word_index
for word, i in sorted(token.word_index.items(),key=lambda x:x[1]):
    if i > (max_features+1):
        break
    try:
        embedding_vector = ft[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix_fasttext[i] = embedding_vector
    except:
        pass

In [132]:
embedding_matrix_fasttext.shape

(5001, 50)

In [133]:
len(word_index)

0

In [134]:
# A simple LSTM with glove embeddings and one dense layer
model = Sequential()
model.add(Embedding(max_features+1,
                    embedding_vector_length, ### 50 here
                    weights=[embedding_matrix_fasttext],
                    input_length=maxlen, ### 1400 here
                    trainable=False))

model.add(LSTM(100, dropout=0.3, recurrent_dropout=0.3))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1400, 50)          250050    
                                                                 
 lstm_2 (LSTM)               (None, 100)               60400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 310,551
Trainable params: 60,501
Non-trainable params: 250,050
_________________________________________________________________


In [135]:
X_train, X_test, y_train, y_test = train_test_split(train.Text_data.values, train.Label.values, 
                                                  stratify=train.Label.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

In [144]:
import tensorflow 
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
token.fit_on_texts(list(X_train) + list(X_test))
X_train_seq = token.texts_to_sequences(X_train)
X_test_seq = token.texts_to_sequences(X_test)

#zero pad the sequences
X_train_pad =pad_sequences(X_train_seq, maxlen=maxlen)
X_test_pad = pad_sequences(X_test_seq, maxlen=maxlen)

In [148]:
def roc_auc(predictions,target):
    '''
    This methods returns the AUC Score when given the Predictions
    and Labels
    '''
    
    fpr, tpr, thresholds = metrics.roc_curve(target, predictions)
    roc_auc = metrics.auc(fpr, tpr)
    return roc_auc

In [145]:
history=model.fit(X_train_pad,y_train,
          epochs=10,
          batch_size=32,          
          validation_data=(X_test_pad, y_test))

Epoch 1/10
223/223 [==============================] - 1459s 7s/step - loss: 0.3969 - accuracy: 0.6734 - val_loss: 0.3794 - val_accuracy: 0.6751
Epoch 2/10
223/223 [==============================] - 1461s 7s/step - loss: 0.3736 - accuracy: 0.6753 - val_loss: 0.3701 - val_accuracy: 0.6751
Epoch 3/10
223/223 [==============================] - 4223s 19s/step - loss: 0.3729 - accuracy: 0.6753 - val_loss: 0.3704 - val_accuracy: 0.6751
Epoch 4/10
223/223 [==============================] - 1753s 8s/step - loss: 0.3715 - accuracy: 0.6753 - val_loss: 0.3720 - val_accuracy: 0.6751
Epoch 5/10
223/223 [==============================] - 1754s 8s/step - loss: 0.3726 - accuracy: 0.6753 - val_loss: 0.3698 - val_accuracy: 0.6751
Epoch 6/10
223/223 [==============================] - 1749s 8s/step - loss: 0.3726 - accuracy: 0.6753 - val_loss: 0.3727 - val_accuracy: 0.6751
Epoch 7/10
223/223 [==============================] - 6820s 31s/step - loss: 0.3721 - accuracy: 0.6753 - val_loss: 0.3710 - val_accurac

In [149]:

test_pred = model.predict(X_test_pad)


56/56 [==============================] - 61s 1s/step


In [152]:
test_pred=test_pred.reshape((test_pred.shape[0],))

In [153]:
test_pred=np.where(test_pred>0.5,1,0)

In [154]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(y_test,test_pred))
print(classification_report(y_test,test_pred))

array([[   0,  397,    0],
       [   0, 1201,    0],
       [   0,  181,    0]], dtype=int64)
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       397
           1       0.68      1.00      0.81      1201
           2       0.00      0.00      0.00       181

    accuracy                           0.68      1779
   macro avg       0.23      0.33      0.27      1779
weighted avg       0.46      0.68      0.54      1779

